In [65]:
import pandas as pd 
import numpy as np
import seaborn as sns
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense,Dropout
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')
sns.set()   #important libraries

[nltk_data] Downloading package punkt to /Users/pratik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [66]:
data=pd.read_csv("/Users/pratik/Desktop/Sentiment_Analysis/Sentiment-Analysis-with-different-NLP-models/IMDB.csv")
  #reading csv file 

In [67]:
collection=[]
for sentence in data['review']:
    words=[word.lower() for word in word_tokenize(sentence)]
    collection.append(words)

num_words=len(collection)    #tokenization of words and storing them in collection

In [68]:
train_size=int(data.shape[0]* 0.8)
X_train= data.review[:train_size]
Y_train= data.sentiment[:train_size]

X_test= data.review[train_size:]
Y_test= data.sentiment[train_size:]  #splitting into the train and test

In [69]:
tokenize=Tokenizer(num_words)
tokenize.fit_on_texts(X_train)
X_train=tokenize.texts_to_sequences(X_train)
X_train=pad_sequences(X_train,maxlen=128,truncating='post',padding='post')  
X_test= tokenize.texts_to_sequences(X_test)
X_test=pad_sequences(X_test,maxlen=128,truncating='post',padding='post') #tokenizer model for adding index to each word and setting
  #up the fixed length by doing padding 

In [70]:
X_train[0],len(X_train[0])

(array([   27,     4,     1,    80,  2102,    45,  1073,    12,   100,
          147,    39,   316,  2968,   409,   459,    26,  3173,    33,
           23,   200,    14,    11,     6,   614,    48,   606,    16,
           68,     7,     7,     1,    87,   148,    12,  3256,    68,
           41,  2968,    13,    92,  5626,     2, 16202,   134,     4,
          569,    60,   271,     8,   200,    36,     1,   673,   139,
         1712,    68,    11,     6,    21,     3,   118,    15,     1,
         7870,  2257,    38, 11540,    11,   118,  2495,    54,  5662,
           16,  5182,     5,  1438,   377,    38,   569,    92,     6,
         3730,     8,     1,   360,   353,     4,     1,   673,     7,
            7,     9,     6,   431,  2968,    14,    12,     6,     1,
        11736,   356,     5,     1, 14689,  6526,  2594,  1087,     9,
         2661,  1432,    20, 22583,   534,    32,  4795,  2451,     4,
            1,  1193,   117,    29,     1,  6893,    25,  2874, 12191,
      

In [23]:
encoder=LabelEncoder()     # used for labelling with numerical values
Y_train= encoder.fit_transform(Y_train)
Y_test = encoder.transform(Y_test)

In [31]:
model = Sequential()    #sequential model for lstm based model generation
model.add(Embedding(input_dim =num_words,output_dim=100,input_length=128,trainable=True))
model.add(LSTM(100,dropout=0.1,return_sequences=True))
model.add(LSTM(100,dropout=0.1))
model.add(Dense(1,activation='sigmoid'))  #giving output between 0 and 1
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [32]:
model.summary()      #summary to watchout the model 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 128, 100)          5000000   
                                                                 
 lstm (LSTM)                 (None, 128, 100)          80400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 5,160,901
Trainable params: 5,160,901
Non-trainable params: 0
_________________________________________________________________


In [33]:
returned=model.fit(X_train,Y_train,epochs=5,batch_size=64,validation_data=(X_test,Y_test))  #giving me the history of accuracy
#fter every epoch what will be the result

Epoch 1/5


2023-05-13 22:29:10.850502: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


625/625 [==============================] - 114s 180ms/step - loss: 0.4595 - accuracy: 0.7837 - val_loss: 0.3587 - val_accuracy: 0.8472
Epoch 2/5
625/625 [==============================] - 112s 179ms/step - loss: 0.2608 - accuracy: 0.9024 - val_loss: 0.3327 - val_accuracy: 0.8582
Epoch 3/5
625/625 [==============================] - 105s 169ms/step - loss: 0.1604 - accuracy: 0.9442 - val_loss: 0.4307 - val_accuracy: 0.8547
Epoch 4/5
625/625 [==============================] - 105s 168ms/step - loss: 0.1050 - accuracy: 0.9649 - val_loss: 0.4330 - val_accuracy: 0.8437
Epoch 5/5
625/625 [==============================] - 106s 170ms/step - loss: 0.0799 - accuracy: 0.9748 - val_loss: 0.5322 - val_accuracy: 0.8454


In [52]:
#base model for some real-life review example 

input_sentence=['The movie is worst']
input_sentence_tokenized=tokenize.texts_to_sequences(input_sentence)
input_sentence_padded=pad_sequences(input_sentence_tokenized,maxlen=128,truncating='post',padding='post')

result=model.predict(input_sentence_padded)
if(result<0.75):
    print("Probablity of Positive=",result)
    print("Negative")
else:
    print("Probablity of Positive=",result)
    print("Positive")

1/1 [==============================] - 0s 25ms/step
Probablity of Positive= [[0.01785491]]
Negative


In [53]:
input_sentence=['The movie is amazing']
input_sentence_tokenized=tokenize.texts_to_sequences(input_sentence)
input_sentence_padded=pad_sequences(input_sentence_tokenized,maxlen=128,truncating='post',padding='post')

result=model.predict(input_sentence_padded)
if(result<0.75):
    print("Probablity of Positive=",result)
    print("Negative")
else:
    print("Probablity of Positive=",result)
    print("Positive")

1/1 [==============================] - 0s 23ms/step
Probablity of Positive= [[0.99823385]]
Positive
